# Assignment 1 (38 marks total)
### Due: March 8, 2024, at 11:59pm

Global rules: you are allowed to add as many code and markdown cells as you would like (and you probably should add some). Your are _not_ allowed to use `scikit-learn` or any other machine learning libraries. There should be enough comments and markdown cells explaining code that it is easy for a reader to understand your code and its outputs.

You will likely want to know about the function `np.linalg.pinv`, which implements the matrix pseudo-inverse, and that the symbol '`@`' is used by numpy to represent matrix multiplication.

## Part 1 - Linear Regression (21 marks total)

In this part, implement linear regression using the closed-form solution for the optimal weights that we saw in class. Next, implement linear regression by finding weights through stochastic gradient descent.

Before you begin, which of your implementations do you think will perform better? Why? Answer this question in the markdown cell below. Note: there is not a correct answer to this question, your evaluation will be based on your explanation (2 marks).

I think the closed form solution will perform better since it requires a single set of matrix operations to get a perfect solution. The stochastic gradient descent algorithm has simpler calculations to perform, but they may need to be performed thousands of times to approach the accuracy of the closed form solution.

Put your implementations below. I have included imports you will almost certainly want, as well as seeded numpy's random number generator to ensure that I get the same output as you when marking. (8 marks)

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

np.random.seed(42)

def linear_regression_closed_form(X, t):
    X = np.concatenate((np.ones(X.shape[0], int)[np.newaxis, :].T, X), axis=1)
    w = np.linalg.pinv(X) @ t

    def predict(xfit):
        xfit = np.concatenate((np.ones(xfit.shape[0], int)[np.newaxis, :].T, xfit), axis=1)
        return [w.T@x for x in xfit]
        
    return predict

def linear_regression_gradient_descent(X, y, epochs, learning_rate):
    X = np.concatenate((np.ones(X.shape[0], int)[np.newaxis, :].T, X), axis=1)
    n_samples, n_features = X.shape
    w = np.zeros(n_features, int)

    indices = np.arange(n_samples)
    np.random.shuffle(indices)

    for epoch in range(epochs):
        for i in indices:
            e = y[i] - np.dot(X[i], w)
            w = w + learning_rate * e * X[i]

    def predict(xfit):
        xfit = np.concatenate((np.ones(xfit.shape[0], int)[np.newaxis, :].T, xfit), axis=1)
        return [w.T@x for x in xfit]
    
    return predict

def get_mse(X, y, model):
    predictions = model(X)
    return np.mean((y - predictions)**2)
    

ModuleNotFoundError: No module named 'numpy'

Next, generate datasets to test your implementations. Use 1-dimensional datasets to visualize the functions estimated by each of your implementations and higher-dimensional datasets to gather numerical data for the validation errors and training times. (9 marks)

In [ ]:
rng = np.random.RandomState(1)
DOMAIN = 10

#1d test
x = DOMAIN*rng.rand(50)
y = -3*x - 15 + 3*rng.rand(50)
plt.scatter(x, y)

#Test Closed Form
model = linear_regression_closed_form(x[:, np.newaxis], y)

xfit = np.linspace(0, 10, 1000)
yfit = model(xfit[:, np.newaxis])

plt.plot(xfit, yfit)

#Test Stochastic Gradient Descent
model = linear_regression_gradient_descent(x[:, np.newaxis], y, 1000, 0.001)

xfit = np.linspace(0, DOMAIN, 1000)
yfit = model(xfit[:, np.newaxis])

plt.plot(xfit, yfit)

In [ ]:
import time
rng = np.random.RandomState(1)

#many dimension test
x = 10*rng.rand(1000, 100)
y = 0.1*rng.rand(100) @ x.T + 15 + 2*rng.rand(1000)

#Test Closed Form
start_time = time.time()
model = linear_regression_closed_form(x, y)
end_time = time.time()
elapsed_time_closed_form = end_time - start_time
print(f"Closed form took: {elapsed_time_closed_form} seconds with mean squared error: {get_mse(x, y, model)}")

#Test Stochastic Gradient Descent
start_time = time.time()
model = linear_regression_gradient_descent(x, y, 50, 0.0001)
end_time = time.time()
elapsed_time_closed_form = end_time - start_time
print(f"Closed form took: {elapsed_time_closed_form} seconds with mean squared error: {get_mse(x, y, model)}")


## Part 2 - Nonlinear Regression (17 marks total)

Using whichever of your linear regresion implementations you found was better in part 1, implement nonlinear regression using Gaussian RBF's. There are many hyperparameters here, so you should use some reasonable deterministic method to choose some of them and use k-fold validation to search for the remaining hyperparameters. Again, generate datasets to test your implementation. Use 1-dimensional datasets to visualize the function predicted by your implementation and use higher-dimensional datasets to gather numerical data for validation errors and training times.

- 6 marks for nonlinear regression implementation
- 6 marks for hyperparameter selection implementation
- 4 marks for testing performance (visualization and numerical)

In [129]:
rng = np.random.RandomState(1)

def rbf_kernel(X, centers, width):
    n_x = X.shape[0]
    n_centers = centers.shape[0]
    
    # Initialize an empty matrix to store kernel values
    kernel_matrix = np.zeros((n_x, n_centers))
    
    # Compute pairwise distances and apply RBF kernel
    for i in range(n_x):
        for j in range(n_centers):
            distance = np.linalg.norm(X[i] - centers[j])
            kernel_matrix[i, j] = np.exp(-(distance**2) / (width**2))
    
    return kernel_matrix

#shuffles the indices of X and divides them into k folds, returning one fold as train_indices and the rest as val_indices every time it is called
def k_fold_cross_validation(X, k = 10):
    indices = np.arange(X.shape[0])
    rng.shuffle(indices)

    folds = np.array_split(indices, k)
    for i in range(k):
        val_indices = folds[i]
        train_indices = np.concatenate([folds[j] for j in range(k) if j != i])
        yield val_indices, train_indices

#trains a model on dataset X and t. domain is the domain of X in all dimensions.
#returns a predict function that, when called with a dataset, returns the y predictions of the model
def gaussian_rbf_regression(X, t, domain):
    def train_model(X, t, domain, width, rbf_count_multiplier):
        num_rbfs = int(X.shape[0]*rbf_count_multiplier)
        linear_model = linear_regression_closed_form(X, t)
        new_t = t - linear_model(X)
    
        centers = np.linspace(0, domain, num_rbfs)
        gaussian_weights = np.linalg.pinv(rbf_kernel(X, centers, width)) @ new_t
    
        def predict(xfit):
            yfit = linear_model(xfit)
        
            yfit += gaussian_weights @ rbf_kernel(xfit, centers, width).T
            
            return yfit
        
        return predict
        
    #Find optimal width and number of rbfs with k-fold cross validation
    widths = np.linspace(0.1, 2, 10)
    rbf_count_multipliers = np.linspace(0.1, 1, 10) #This multiplied by the number of samples rounded to an int gives the number of rbfs
    num_folds = 4

    mse_scores = []
    
    for width in widths:
        rbf_count_multiplier_mse_scores = []
        for rbf_count_multiplier in rbf_count_multipliers:
            fold_mse_scores = []
            
            for train_index, val_index in k_fold_cross_validation(X, num_folds):
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]

                model = train_model(X_train, y_train, domain, width, rbf_count_multiplier)
                mse = get_mse(X_val, y_val, model)
                fold_mse_scores.append(mse)

            avg_mse = np.mean(fold_mse_scores)
            rbf_count_multiplier_mse_scores.append(avg_mse)

        mse_scores.append(rbf_count_multiplier_mse_scores)

    mse_scores = np.array(mse_scores)
    optimal_width_index, optimal_rbf_count_multiplier_index = np.unravel_index(np.argmin(mse_scores, axis=None), mse_scores.shape)
    optimal_rbf_count_multiplier = rbf_count_multipliers[optimal_rbf_count_multiplier_index]
    optimal_width = widths[optimal_width_index]
    
    model = train_model(X, t, domain, optimal_width, optimal_rbf_count_multiplier)
    return model

In [ ]:
rng = np.random.RandomState(9)

#1D Test
DOMAIN = 20
n = 200
dim = 1

num_rbfs = 5
centers = DOMAIN*rng.rand(num_rbfs, dim)
width = 1
gaussian_weights = 10*rng.rand(num_rbfs) - 5

x = DOMAIN*rng.rand(n)
y = x/2 + 2*rng.rand(n) + gaussian_weights @ rbf_kernel(x[:, np.newaxis], centers, width).T

model = gaussian_rbf_regression(x[:, np.newaxis], y, DOMAIN)
plt.scatter(x, y);
xfit = np.linspace(min(x), max(x), 1000)
yfit = model(xfit[:, np.newaxis])
plt.plot(xfit, yfit)
print(f"Mean Squared Error: {get_mse(x[:, np.newaxis], y, model)}")

In [ ]:
import time
rng = np.random.RandomState(1)

#Many Dimension Test (this takes about 70 seconds)
DOMAIN = 20
n = 500
dim = 50

num_rbfs = 10
centers = DOMAIN*rng.rand(num_rbfs, dim)
width = 5*rng.rand()
gaussian_weights = 10*rng.rand(num_rbfs) - 5

x = 10*rng.rand(n, dim)
y = 0.1*rng.rand(dim) @ x.T + 15 + 2*rng.rand(n) + gaussian_weights @ rbf_kernel(x, centers, width).T

start_time = time.time()
model = gaussian_rbf_regression(x, y, DOMAIN)
end_time = time.time()
elapsed_time_closed_form = end_time - start_time
print(f"Closed form took: {elapsed_time_closed_form} seconds with mean squared error: {get_mse(x, y, model)}")

How well do you think your implementation is performing? Why? Answer in the markdown cell below, referring to the results of your tests above. (1 mark)

With mean squared errors of around 0.2, I think my implementation is fairly accurate but based on the visual representation of the one dimensional test, I think my basis functions are overfitting slightly. My implementation is slower than I would have liked, and I think this is coming from the nested loops in the rbf kernel function. I initially had an implementation that used exclusively matrix operations and it performed much faster on the one dimensional test, but I wasn't able to get it to work with multidimensional data.